<a href="https://colab.research.google.com/github/DiggsPapu/data-science/blob/lab2/Lab2TensorFlowMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Red Neuronal Profunda (DNN) para clasificación MNIST

Aplicaremos todos nuestros conocimientos para crear una DNN, frecuentemente llamada también una Artificial Neural Network (ANN).  El problema que vamos a trabajar se conoce como el "Hola Mundo" del aprendizaje profundo porque para la mayoría de estudiantes este es el primer algoritmo de aprendizaje profundo que ven.

El conjunto de datos se llama MNIST y se refiere al reconocimiento de dígitos escritos a mano.  Pueden encontrar más información en el sitio web de Yann LeCun (Director of AI Research, Facebook).  El es uno de los pioneros de todo este tema, así como de otras metodologías más complejas como las Redes Neurales Convolucionales (CNN) que se utilizan hoy día.

El conjunto de datos tiene 70,000 imágenes (28x28 pixels) de dígitos escritos a mano (1 dígito por imagen).

La meta es escribir un algoritmo que detecta qué dígito ha sido escrito.  Como solo hay 10 dígitos (0 al 9), este es un problema de clasificación con 10 clases.

Nuestra meta será construir una RN con 2 capas escondidas.

## Importar los paquetes relevantes

TensorFlow incluye un proveedor de datos de MNIST que utilizaremos acá.  Viene con el módulo **"tensorflow.keras.datasets"**.

In [58]:
import numpy as np
import tensorflow as tf

La siguiente instrucción, cuando se corre por primera vez, descarga el conjunto de datos en lo indicado por el parámetro path, relativo a  ~/.keras/datasets).  Como si se hubiera ejecutado Lo siguiente:

tf.keras.datasets.mnist.load_data(
    path='mnist.npz'
)

luego separa los datos en un conjunto para entrenamiento y otro para pruebas.

Si se ejecuta más de una vez, ya no descarga el archivo.

In [59]:
(X_entreno, y_entreno), (X_prueba, y_prueba) = tf.keras.datasets.mnist.load_data()
assert X_entreno.shape == (60000, 28, 28)
assert X_prueba.shape == (10000, 28, 28)
assert y_entreno.shape == (60000,)
assert y_prueba.shape == (10000,)

## Datos

Esta sección es donde pre-procesaremos nuestros datos.

Por default, TF2 tiene conjuntos de datos de entrenamiento y de prueba, pero no tiene un conjunto de validación, por lo que debemos dividirlo por nuestra cuenta

Lo haremos del mismo tamaño que el conjunto de prueba

In [60]:
num_obs_validacion = y_prueba.shape[0]

Usaremos una variable dedicada para el número de muestras de prueba

In [61]:
num_obs_prueba = y_prueba.shape[0]

Generalmente preferimos "normalizar" nuestros datos en alguna forma para que el resultado sea numéricamente más estable.  En este caso simplemente preferimos tener entradas entre 0 y 1, por lo que definimos una función, que reciba la imagen MNIST.

Como los posibles valores de las entradas son entre 0 y 255 (256 posibles tonos de gris), al dividirlos por 255 obtenemos el resultado deseado.

In [62]:
X_entreno_normalizado = X_entreno / 255

Finalmente, normalizaremos y convertiremos los datos de pruebas en tandas.  Los normalizamos para que tengan la misma magnitud que los datos de entrenamiento y validación.

No hay necesidad de "barajearlo" ya que no estaremos entrenando con los datos de prueba.  Habra una sola tanda, igual al tamaño de los datos de prueba.

In [63]:
X_prueba_normalizado = X_prueba / 255

Una vez se han "normalizado" los datos, podemos proceder a extraer los datos de entrenamiento y de validación.

Nuestros datos de validación serán 10000 para ser igual al conjunto de prueba.

Finalmente, creamos una tanda con un tamaño de tanda igual al total de muestras de validación.

In [64]:
X_validacion = X_entreno_normalizado[-num_obs_validacion: , : , : ]
y_validacion = y_entreno[-num_obs_validacion:]

Similarmente, los datos de entrenamiento son todos los demás por lo que nos salteamos tantas observaciones como las hay en el conjunto de validación.

In [65]:
X_entreno = X_entreno_normalizado[ : X_entreno_normalizado.shape[0] - num_obs_validacion, : , : ]
y_entreno = y_entreno[ : y_entreno.shape[0] - num_obs_validacion]
num_obs_entreno = y_entreno.shape[0]

Convertir de Arreglos Numpy a Tensores

In [66]:
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

Barajear y hacer tandas con el conjunto de datos de entrenamiento

In [67]:
TAMANIO_TANDA = 700
datos_entreno = datos_entreno.shuffle(buffer_size = num_obs_entreno).batch(TAMANIO_TANDA)

Hacer tandas con los conjunto de validación y prueba, no se necesita barajearlos

In [68]:
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

In [69]:
datos_validacion, datos_prueba


(<_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>,
 <_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>)

## Modelo

### Delineamos el modelo

Cuando pensamos sobre un algoritmo de aprenzaje profundo, casi siempre solo lo imaginamos.  Asi que esta vez, hagámoslo.  :)

In [70]:
tamanio_entrada = 784
tamanio_salida = 10

Usaremos el mismo ancho para ambas capas escondidas.  No es una necesidad!

In [71]:
tamanio_capa_escondida = 3000

# Definimos cómo se verá el modelo

La primera capa (la de entrada):  cada observación es de 28x28 píxeles, por lo tanto es un tensor de rango 2.

Como aún no hemos aprendido sobre CNNs, no sabemos como alimentar este tipo de entrada a nuestra red, por lo tanto hay que "aplanar" las imágenes.  Hay un método conveniente **Flatten** que toma nuestro tensor de 28x28 y lo convierte en  un vector (None), o (784,)...porque 28x28 = 784.  Esto nos permite crear una red de alimentación hacia adelante.

    
**tf.keras.layers.Dense** básicamente implementa:  output = activation(dot(entrada, peso) + sesgo).  Requiere varios argumentos, pero los más importantes para nosotros son el ancho de la capa escondida y la función de activación.

La capa final no es diferente, solo nos aseguramos de activarla con **softmax**


In [72]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada

    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 3ra capa escondida
    # tf.keras.layers.Dense(tamanio_capa_escondida, activation='tanh'), # 4ta capa escondida
    # tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 5ta capa escondida
    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

### Seleccionar el optimizador y la función de pérdida

Definimos el optimizador que nos gustaría utilizar, la función de pérdida, y las métricas que nos interesa obtener en cada interacción

In [73]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Entrenamiento

Acá es donde entrenamos el modelo que hemos construído

Determinamos el número máximo de épocas.

Ajustamos el modelo , especificando:

* los datos de entrenamiento
* el número total de épocas
* y los datos de validación que creamos en el formato (entradas, metas)

In [74]:
NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS,
          validation_data = datos_validacion,
          verbose = 2)

Epoch 1/5
72/72 - 156s - 2s/step - accuracy: 0.8894 - loss: 0.3628 - val_accuracy: 0.9695 - val_loss: 0.1051
Epoch 2/5
72/72 - 151s - 2s/step - accuracy: 0.9760 - loss: 0.0774 - val_accuracy: 0.9745 - val_loss: 0.0872
Epoch 3/5
72/72 - 202s - 3s/step - accuracy: 0.9854 - loss: 0.0468 - val_accuracy: 0.9770 - val_loss: 0.0819
Epoch 4/5
72/72 - 205s - 3s/step - accuracy: 0.9893 - loss: 0.0326 - val_accuracy: 0.9733 - val_loss: 0.0955
Epoch 5/5
72/72 - 151s - 2s/step - accuracy: 0.9931 - loss: 0.0219 - val_accuracy: 0.9760 - val_loss: 0.0963


## Probar el modelo

Como se discutió en clase, luego del entrenamiento (con los datos de entrenamiento), y la validación (con los datos de validación), probamos el potencial de predicción final de nuestro modelo con el conjunto de datos de prueba que el algoritmo NUNCA ha visto antes.

Es muy importante reconocer que estar "jugando" con los hiperparámetros sobre-ajusta el conjunto de datos de validación.

La prueba es la instancia absolutamente final. Nunca debe probarse el modelo antes de haber completamente ajustado el modelo.

Si se ajusta el modelo después de hacer la prueba, se empezará a sobre-ajustar el conjunto de datos de prueba, que echaría "por los suelos" el propósito original del mismo.

In [75]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

15/15 ━━━━━━━━━━━━━━━━━━━━ 9s 574ms/step - accuracy: 0.9747 - loss: 19.4578


In [76]:
# Si se desea, se puede aplicar un formateo "bonito"
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Pérdida de prueba: 16.21. Precisión de prueba: 97.90%


In [77]:
# -*- coding: utf-8 -*-
"""Lab2 DS.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/13FXF4kuaRooPwRQyD23hvvICtJGJ1dB_
"""

# Importar las bibliotecas necesarias
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD, Adam
import time

# Cargar los datos de MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Crear el modelo base
def create_model():
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    return model

# Compilar y entrenar el modelo con diferentes configuraciones
def train_model(model, batch_size, learning_rate, epochs=5):
    model.compile(optimizer=SGD(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    start_time = time.time()
    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
                        validation_data=(x_test, y_test))
    end_time = time.time()
    training_time = end_time - start_time
    return history, training_time

# Línea base: Tamaño de tanda 32, tasa de aprendizaje 0.01
model_base = create_model()
history_base, time_base = train_model(model_base, batch_size=32, learning_rate=0.01)
print("Línea base - Precisión de validación:", max(history_base.history['val_accuracy']))
print("Línea base - Tiempo de entrenamiento:", time_base)

# 6. Ajuste del tamaño de la tanda a 10,000
model = create_model()
history, training_time = train_model(model, batch_size=10000, learning_rate=0.01)
print("\nBatch size 10,000 - Precisión de validación:", max(history.history['val_accuracy']))
print("Batch size 10,000 - Tiempo de entrenamiento:", training_time)

# 7. Ajuste del tamaño de la tanda a 1 (SGD)
model = create_model()
history, training_time = train_model(model, batch_size=1, learning_rate=0.01)
print("\nBatch size 1 (SGD) - Precisión de validación:", max(history.history['val_accuracy']))
print("Batch size 1 (SGD) - Tiempo de entrenamiento:", training_time)

# 8. Ajuste de la tasa de aprendizaje a 0.0001
model = create_model()
history, training_time = train_model(model, batch_size=32, learning_rate=0.0001)
print("\nLearning rate 0.0001 - Precisión de validación:", max(history.history['val_accuracy']))
print("Learning rate 0.0001 - Tiempo de entrenamiento:", training_time)

# 9. Ajuste de la tasa de aprendizaje a 0.02
model = create_model()
history, training_time = train_model(model, batch_size=32, learning_rate=0.02)
print("\nLearning rate 0.02 - Precisión de validación:", max(history.history['val_accuracy']))
print("Learning rate 0.02 - Tiempo de entrenamiento:", training_time)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.7358 - loss: 1.0332 - val_accuracy: 0.9062 - val_loss: 0.3533
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.9052 - loss: 0.3492 - val_accuracy: 0.9194 - val_loss: 0.2891
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9183 - loss: 0.2920 - val_accuracy: 0.9294 - val_loss: 0.2576
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9267 - loss: 0.2626 - val_accuracy: 0.9342 - val_loss: 0.2360
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9338 - loss: 0.2363 - val_accuracy: 0.9373 - val_loss: 0.2174
Línea base - Precisión de validación: 0.9373000264167786
Línea base - Tiempo de entrenamiento: 37.53990840911865
Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step - accuracy: 0.0674 - loss: 2.3820 - val_accuracy: 0.0993 - val_loss: 2.3101
Epoch 2/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - accuracy: 0.1026 - loss: 2.3054 - val_accuracy: 0.1526 - val_loss:

Utilizando el modelo inicial y los hiperparámetros dados en este notebook, la precisión de prueba final debe ser aproximadamente 97%.

Cada vez que se ejecuta el código, se obtiene una precisión diferente debido a la "barajeada" de las tandas, los pesos se inicializan en forma diferente, etc.

Finalmente, intencionalmente se ha llegado a una solución subóptima, para que puedan tener la oportunidad de mejorarla como ejercicio de laboratorio.